In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [2]:
#@title Helper functions
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))

    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

## Dataset

In [3]:
X,Y = load_coffee_data();
print(X.shape, Y.shape)

(200, 2) (200, 1)


## Normalize the data

In [4]:
norm_l = tf.keras.layers.Normalization(axis = -1)
norm_l.adapt(X)
Xn = norm_l(X) # Normalized X

In [5]:
# Tile data to increase training set size and decrease the number of training epochs
X_train = np.tile(Xn, (1000, 1))
Y_train = np.tile(Y, (1000, 1))
print(X_train.shape, Y_train.shape)

(200000, 2) (200000, 1)


## Tensorflow Model

### Model

In [6]:
tf.random.set_seed(1234)
model = tf.keras.models.Sequential ([
    tf.keras.Input(shape = (2, )),
    tf.keras.layers.Dense(3, activation = 'sigmoid', name = 'layer1'),
    tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'layer2')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13 (52.00 Byte)
Trainable params: 13 (52.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
def print_weights():
    W1, b1 = model.get_layer('layer1').get_weights()
    W2, b2 = model.get_layer('layer2').get_weights()
    print(f"W1 {W1.shape}:\n", W1, f"\nb1 {b1.shape}:\n", b1)
    print(f"W2 {W2.shape}:\n", W2, f"\nb2 {b2.shape}:\n", b2)
print_weights()

### Model compile and Model fit

In [8]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
)

model.fit(X_train, Y_train, epochs = 10)

Epoch 1/10
6250/6250 [==============================] - 11s 2ms/step - loss: 0.2017
Epoch 2/10
6250/6250 [==============================] - 11s 2ms/step - loss: 0.1181
Epoch 3/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0894
Epoch 4/10
6250/6250 [==============================] - 10s 2ms/step - loss: 0.0213
Epoch 5/10
6250/6250 [==============================] - 13s 2ms/step - loss: 0.0120
Epoch 6/10
6250/6250 [==============================] - 11s 2ms/step - loss: 0.0080
Epoch 7/10
6250/6250 [==============================] - 9s 1ms/step - loss: 0.0055
Epoch 8/10
6250/6250 [==============================] - 11s 2ms/step - loss: 0.0038
Epoch 9/10
6250/6250 [==============================] - 11s 2ms/step - loss: 0.0027
Epoch 10/10
6250/6250 [==============================] - 12s 2ms/step - loss: 0.0019


### Updated weights

In [15]:
print_weights()

W1 (2, 3):
 [[  0.15976745  14.604166   -10.956867  ]
 [ 10.412597    12.105809    -0.30317226]] 
b1 (3,):
 [ 12.572174    1.9825191 -11.779441 ]
W2 (3, 1):
 [[ 42.586227]
 [-44.72363 ]
 [-52.13249 ]] 
b2 (1,):
 [-13.182875]


### Predictions

In [16]:
X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 [==============================] - 0s 115ms/step
predictions = 
 [[9.9190062e-01]
 [2.1646481e-07]]


In [33]:
# Applying a threshold

yhat = (predictions >= 0.5).astype(int)
print(f"Predictions: \n", np.r_[[['X1', 'X2', 'Y_hat']], np.c_[X_test, yhat]])

Predictions: 
 [['X1' 'X2' 'Y_hat']
 ['200.0' '13.9' '1.0']
 ['200.0' '17.0' '0.0']]
